<a href="https://colab.research.google.com/github/IsabellaBonnet/analitica/blob/main/2_Modelo_Arbol_Clase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Arboles de decisión**

0. Cargar las librerias de referencia para el modelo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #Graficador de cosas
from sklearn.tree import DecisionTreeClassifier

#Para no tener que cargar el archivo
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. se procede con la carga de los datos del archivo SolicitantesCrédito

In [ ]:
nxl='/content/drive/MyDrive/ACTUAL SEMESTRE/ANÁLITICA DE NEGOCIOS /Archivos /SolicitantesCrédito(USD)-2.xlsx'
XDB=pd.read_excel(nxl,sheet_name=1)
XDB=XDB.dropna()
XDB.head(100)

#Se procede con la selección de las variables de trabajo
XD=np.array(XDB[['Edad','Ingresos','Egresos','Monto (EAD)']]) #datos entrada #Fixed typo: added closing parenthesis
print(XD)
yd=np.array(XDB['PreApr']) #datos salida, el dato que quiero que el modelo pronostique
print(yd)

[[  38.         1356.144      1685.62266667  625.56223022]
 [  51.          286.016       364.462       140.03198442]
 [  37.          517.46325     629.20888889  284.56449156]
 ...
 [  38.          773.01975     672.91066667  594.94789369]
 [  43.          635.50175     780.69155556  305.58053861]
 [  37.         1419.26675    1169.83844444 1005.63687051]]
[1 1 0 ... 1 1 1]


2. Creación del modelo-decision Tree



In [ ]:
mar=DecisionTreeClassifier(criterion='gini',max_depth=4) #4 niveles o variables de entrada
mar.fit(XD,yd)
ypd=mar.predict(XD)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(yd,ypd)
print(cm)

#Eavaluamos el modelo
VN=cm[0][0]
FP=cm[0][1]
FN=cm[1][0]
VP=cm[1][1]
Ex= (VN+VP)/(VN+VP+FN+FP) #que porcentaje de los datos pronosticó correctamente VN y VP es cuantos les pegó el modelo
print('La exactitud alcanzada por el modelo es:', Ex)
Se=VP/(FN+VP) #como le fue con los positivos
print('La sensibilidad alcanzada por el modelo es:', Se)
Es=VN/(VN+FP) #como le fue con los negativos
print('La especificidad alcanzada por el modelo es:', Es)


[[2301  658]
 [ 644 2239]]
La exactitud alcanzada por el modelo es: 0.7771311194796303
La sensibilidad alcanzada por el modelo es: 0.7766215747485259
La especificidad alcanzada por el modelo es: 0.7776275768840825


3. ¿Y dónde está el Árbol?

In [ ]:
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data

vs=['Edad','Ingresos','Egresos','Monto (EAD)'] #son los titulos para los nodos
dot_graph=export_graphviz(mar,feature_names=vs)
graph=graph_from_dot_data(dot_graph)
graph.write_png('arbol.png')

True

4. se procede con el pronóstico de los nuevos solicitantes

In [ ]:
XDB2=pd.read_excel(nxl,sheet_name=2)
XDB2=XDB2.dropna()
XDB2.head(100)
XD2=np.array(XDB2[['Edad','Ingresos','Egresos','Monto (EAD)']])
print(XD2)


#pronosticamos la PreAprobación
ydp2=mar.predict(XD2)
print(ydp2)
print('El numero de PreAprobados es:',np.sum(ydp2))
print('El numero de PreNegados es:',len(ydp2)-np.sum(ydp2))
print('El numero de total de Datos es:',len(ydp2))

[[  38.          383.67775     368.59866667  175.84723507]
 [  47.           34.1875       38.03444444   26.95587069]
 [  46.         1774.305      2116.72377778  705.43470796]
 ...
 [  32.          283.26575     170.26644444  225.30034584]
 [  35.          768.288       847.26444444  479.16423193]
 [  24.          487.598       376.18355556  424.283721  ]]
[0 0 1 ... 0 1 0]
El numero de PreAprobados es: 1260
El numero de PreNegados es: 1276
El numero de total de Datos es: 2536


**Análisis de resultados**

de acuerdo con la base de datos suministrada, se puede observar un total de datos y el nodo base del arbol de decisión  se puede observar un total de 5842 datos, divididos en 2959 Pre Aprobados y un total de 288 PreNegados, esto representa el 51% y el 49% respectivamente.

De acuerdo con el pronóstico realizado por el modelo, este alcanzó una exactitud cercana al 77%, valores que fueron similares a la sensibilidad que mide el comportamiento del modelo frente a la clasificación de los Pre Aprobados, así como frente a la Especificidad, indicador que mide el comportamiento del modelo frente a la clasificación de los Pre Negados.

Se destaca el único Nodo Puro (PreNeg:10, PreApr:0), en donde los solicitantes de crédito deben cumplir con la siguiente regla: SI Ing>644 AND Egr<=685 and Ing>376 and Monto<=322.

Con respecto al pronóstico, se puede observar un total de 2536 individuos sin Aprovación, de los cuales un total de 1260 pueden ser PreAprobados y un total de 1276 que pueden ser PreNegados.